In [ ]:
# from experiments import generate_dataset
from query_processor import QP_sp_continuity
import matplotlib.pyplot as plt
import networkx as nx
import os
import json
from spreadnet.datasets.data_utils.draw import draw_networkx
from spreadnet.utils import post_processor
from query_processor import QP_sp_continuity as qp_utils
import timeit
from copy import deepcopy
%matplotlib inline

In [ ]:
file_2 = os.path.join('predicted_exp1_sp.json')
pred_grp = QP_sp_continuity.read_json_file(file_2)

file_3 = os.path.join('ground_truth_exp1_sp.json')
gt_grp = QP_sp_continuity.read_json_file(file_3)

In [ ]:
gt_grph_1 = QP_sp_continuity.read_json_file('gt_exp2.json')
pred_grp_1 = QP_sp_continuity.read_json_file('pred_exp2.json')

In [ ]:
QP_sp_continuity.draw_networkx_utils(pred_grp)

In [ ]:
QP_sp_continuity.draw_networkx_utils(pred_grp_1)

In [ ]:
def is_continuos(G):
    # find start and nodes
    g_path_idx, g_edges_idx, g_pos = QP_sp_continuity.extract_path(G)


    is_start_index = 0
    is_end_index = 0
    
    print(f"edges: {g_edges_idx}")
    detected = []
    missing_out_edge = []
    for i in g_path_idx:  # list of node indexes that are in path
        if G.nodes[i]["is_start"] is True:
            is_start_index = i
        
        if G.nodes[i]["is_end"] is True:
            is_end_index = i
        
        succ_list = list(G.successors(i))
        print(f"curr_node:{i}, successors:{succ_list}")
        out_from_curr = list(
            G.out_edges(i, data="is_in_path")
        )  # outgoing edges from curr node
       
        missing = 0
        for e in out_from_curr:
            if e[2] == True:  # for each outgoing edge e, 'is_in_path' is true
                print(f"{e} --> edge exits")
                detected.append(e)
                break
            
            else:
                missing = missing + 1
                if missing == len(succ_list):  # iterate over all successor nodes until end node reached
                    if G.nodes[i]["is_end"] is False:
                        print(f"no outgoing edges from node {i} to successors")
                        missing_out_edge.append(i)
                        
                    else:
                        print(f"end node reached:{i}")
                
                        break
                    print("\n")
   
        
                    
    #print("\n\n")
    print(f"nodes_in_path:{g_path_idx}")
    #print(f"Start_node:{is_start_index}")
    #print(f"End_node:{is_end_index}")
    print(f"detected out going edges from nodes that are predicted as 'is_in_path':{detected}")
    print(f"missing out going edge from :{missing_out_edge}")
    
    


In [ ]:
def look_ahead(G, node_idx):
    
    in_cont_path = []
    node_out_edges = list(G.out_edges(node_idx,data=True))
    # print(f"node's_all_edges: {G.edges(node_idx)}\n")
    # print(f"node's_in_edges: {G.in_edges(node_idx)}\n")
    # print(f"node's_out_edges: {G.out_edges(node_idx)}\n")
    
    for u,v,data in node_out_edges:
        # print(u,v)
        # print(data['is_in_path'])
        e_frm_u = data['is_in_path'] 
        next_node_v = G.nodes[v]['is_in_path']
        
        if G.nodes[v]['is_end']:
            in_cont_path.append(tuple((u,v,'end_is_connected')))
            break
        else:
           
            if e_frm_u and next_node_v: 
                    #confirmed link between u,v where uv are also in path
                    # print(f"edge {u} -> {v},    node {v} exists\n")
                in_cont_path.append(tuple((u,v,'is_connected')))
                
            elif e_frm_u==False and next_node_v:
                    # missing link between u,v but v is in path; hence this a potential edge
                    # print(f"edge {u} -- {v} ,   but next node {v} exists\n")
                in_cont_path.append(tuple((u,v,'is_potential_edge')))
                
            elif e_frm_u and next_node_v==False:
                    #confirmed link between u,v but v is not in path; hence this is potential next node 
                    # print(f"edge {u} -> {v}     but next node {v} does not exist\n")
                in_cont_path.append(tuple((u,v,'is_potential_node')))
                
            elif e_frm_u==False and next_node_v==False:
                    #neither edge or node is in path; not required
                    # print(f"edge {u} -- {v}     next node {v} does not exist\n")
                in_cont_path.append(tuple((u,v,'is_disconnected')))       
   
    return in_cont_path
    

In [ ]:
look_ahead(pred_grp,4)

In [ ]:
look_ahead(pred_grp_1,17)

In [ ]:
look_ahead(pred_grp,20)

In [ ]:
look_ahead(pred_grp_1,17)

In [ ]:
pred_grp.edges[20,2]

In [ ]:
def node_walk(G,start_node_idx):
        
    visited = [start_node_idx]
    #visited = []
    # print(f"start:{visited[0]}")
    next_visit = visited[-1]
    #next_visit = [start_node_idx]
    print(f"next_visit:{next_visit}")
    
    potential_node = []
    potential_edge = []
    flag = True
    
    
    while flag and G.nodes[next_visit]['is_in_path']:
            
        in_cont_path = look_ahead(G,next_visit)
        for u,v,status in in_cont_path :
                check_u_in_path = G.nodes[u]['is_in_path']
                check_v_in_path = G.nodes[v]['is_in_path']
                check_e_in_path = G.edges[u,v]['is_in_path']
                if status=='is_connected' or status=='end_is_connected':
                        print(f" is_connected at:{u},{v}")
                        visited.append(v)
                        print(f"visited: {visited}")
                        next_visit = visited[-1]
                        print(f"next_visited: {next_visit}")
                        flag= True
                        break
                
                elif status=='is_potential_node':   
                        potential_node.append(v)
                        print(f"potential node at:{u},{v}")
                        print(f"visited: {visited}")
                        next_visit = visited[-1]
                        print(f"next_visited: {next_visit}")
                        flag= True
                        continue  
                           
                elif status=='is_potential_edge' and check_v_in_path==False:
                        print(f"potential_edge at:{u},{v}")
                        potential_edge.append(v)
                        print(f"visited: {visited}")
                        next_visit = visited[-1]
                        print(f"next_visited: {next_visit}")
                        flag= True
                        continue
                
                # elif status=='end_is_connected':
                #         print(f"end_is_connected at:{u},{v}")
                #         visited.append(v)
                #         print(f"visited: {visited}")
                #         flag= False
                #         next_visit = visited[-1]
                #         print(f"next_visited: {next_visit}")
                #         break
                
                elif status=='is_disconnected':
                        print(f"disconnected at:{u},{v}")
                        print(f"visited: {visited}")
                        flag= False
                        next_visit = visited[-1]
                        print(f"next_visited: {next_visit}")
                        continue 
                     
                        
                        
                        
   
    return visited, potential_node, potential_edge 
        

In [ ]:
node_walk(pred_grp,20)

In [ ]:
node_walk(pred_grp_1,7)

In [ ]:
nx.is_simple_path(gt_grp,node_walk(gt_grp,21)[0])

In [ ]:
visited, potential_node, potential_edge  = node_walk(pred_grp_1,7)
visited, potential_node, potential_edge

In [ ]:
visited, potential_node, potential_edge  = node_walk(pred_grp,20)
visited, potential_node, potential_edge

In [ ]:
def node_walk_1(G,start_node_idx):
    visited = [start_node_idx]
    next_visit = visited[-1]
    
    potential_node = []
    potential_edge = []
    flag = True
    
    while flag and G.nodes[next_visit]['is_in_path']:
        in_cont_path = look_ahead(G,next_visit)
        for u,v,status in in_cont_path :
                check_u_in_path = G.nodes[u]['is_in_path']
                check_v_in_path = G.nodes[v]['is_in_path']
                check_e_in_path = G.edges[u,v]['is_in_path']
                if status=='is_connected' or status=='end_is_connected':
                        print(f" is_connected at:{u},{v}")
                        visited.append(v)
                        print(f"visited: {visited}")
                        next_visit = visited[-1]
                        print(f"next_visited: {next_visit}")
                        flag= True
                        break
                else:
                    if status=='is_potential_node':   
                            potential_node.append(v)
                            print(f"potential node at:{u},{v}")
                            print(f"visited: {visited}")
                            next_visit = visited[-1]
                            print(f"next_visited: {next_visit}")
                            flag= True
                            continue  
                            
                    elif status=='is_potential_edge' and check_v_in_path==False:
                            print(f"potential_edge at:{u},{v}")
                            potential_edge.append(v)
                            print(f"visited: {visited}")
                            next_visit = visited[-1]
                            print(f"next_visited: {next_visit}")
                            flag= True
                            continue
                    
        else:
                if status=='is_disconnected':
                            print(f"disconnected at:{u},{v}")
                            print(f"visited: {visited}")
                            print(f"next_visited: {next_visit}")
                break
                             
  
    return visited, potential_node, potential_edge 
        

In [ ]:
node_walk_1(pred_grp,2)

In [ ]:
node_walk_1(pred_grp_1,16)


In [ ]:
node_walk_1(gt_grp,21)

In [ ]:
def have_bidirectional_relationship(G, node1, node2):
    return G.has_edge(node1, node2) and G.has_edge(node2, node1)


In [ ]:
have_bidirectional_relationship(pred_grp,5,4)

In [ ]:
ex_wlk_1_grph, ex_wlk_1_edge_weights = post_processor.exhaustive_probability_walk(deepcopy(pred_grp),prob_threshold=0.25)
ex_wlk_1_grph,ex_wlk_1_edge_weights
applied_grph,applied_grph_edges = post_processor.apply_path_on_graph(deepcopy(pred_grp),path=ex_wlk_1_edge_weights,require_clean=False)
qp_utils.draw_networkx_utils(applied_grph)

In [ ]:
list(nx.dfs_edges(applied_grph_1[0], source= 21))